# i-Vectors Tutorial

In [ ]:
hpath='/exp/jvillalba/hyperion/hyperion-persephone'
import sys
sys.path.append(hpath)

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import hyperion as hyp
import hyperion.np as hnp

In [ ]:
def generate_data(num_dims, num_spks=10, num_utts=10, num_units=10, unit_length=10, tv_dim=2):
    """ Generate data following the i-vector model

    Args:
      num_dims: number of dimensions of the features.
      num_spks: number of speakers.
      num_utts: number of utterances per speaker.
      num_units: number of phonetic units per utterance.
      unit_length: duration of each phonetic unit.
    """
    rng = np.random.RandomState(seed=1234)
    # we set the number of phonetic classes to 2^num_dim
    num_comp = 2**num_dims
    
    # Define UBM
    # Means of the GMM-UBM
    ubm_means = np.zeros((num_comp, num_dims))
    kernel=np.array([1.,-1.])[:,None]
    ubm_means = kernel
    for i in range(1,num_dims):
        ubm_means = np.concatenate((np.repeat(kernel, int(2**i), axis=0), np.tile(ubm_means,(2,1))), axis=1)
        
    # Covariances of the GMM-UBM
    ubm_cov = 0.1 * np.ones((num_comp, num_dims))
    ubm_prec = 1./ubm_cov

    # Weights of the GMM-UBM
    ubm_weights = np.ones((num_comp))/num_comp
    

    # Define between and within speaker covariances
    sb = 0.7
    sw = 0.3

    # Define Total Variability sub-space
    T = rng.randn(tv_dim, num_dims * num_comp)
    T = 0.2 * T/np.max(T)
    
    # Sample speakers
    spk_ids = np.arange(num_spks)
    y = sb * rng.randn(num_spks, tv_dim)

    # Sample i-vectors
    spk_ids = np.repeat(spk_ids, num_utts, axis=0)
    y = np.repeat(y, num_utts, axis=0)
    w = y + sw * rng.randn(num_spks*num_utts, tv_dim)

    x = []
    r_idx = []
    # Sample features
    for i in range(w.shape[0]):
      # For each utterance
      # Compute the GMM mean of the utterance
      means_i = ubm_means + np.dot(w[i],T).reshape(num_dims,num_comp).T

      # Create a GMM for the utterance.
      gmm = hnp.pdfs.GMMDiagCov(pi=ubm_weights, mu=means_i, Lambda=ubm_prec)

      # Sample the Gaussian components
      r_i = rng.multinomial(1, ubm_weights, size=(num_units,))
      # Assume that we stay in the same component several time steps.
      r_i = np.repeat(r_i, unit_length, axis=0)
      # Draw samples from the GMM
      x_i = gmm.sample(r=r_i)
      x.append(x_i)
      r_idx.append(r_i.argmax(axis=-1))

    return x, r_idx, spk_ids



In [ ]:
x_dim=3
x, r_idx, spk_ids=generate_data(num_dims=x_dim)

In [ ]:
x_cat=np.concatenate(x, axis=0)
fig = plt.figure()
ax = fig.add_subplot(projection='3d')
ax.scatter(x_cat[:,0], x_cat[:,1], x_cat[:,2], marker='o')
ax.set_xlabel('x1')
ax.set_ylabel('x2')
ax.set_zlabel('x3')
plt.show()

In [ ]:
num_comp=8
y_dim=2

In [ ]:
ubm_gmm = hnp.pdfs.GMMDiagCov(num_comp=num_comp, x_dim=x_dim)
elbo, elbo_norm = ubm_gmm.fit(x_cat, epochs=10)
fig = plt.figure()
plt.plot(elbo_norm)
plt.xlabel('iters')
plt.ylabel('log(p(x))')
plt.grid(True)

In [ ]:
ubm_gmm.mu
ubm_gmm.pi
ubm_gmm.Sigma
fig=plt.figure()
ax=fig.add_subplot(111, projection="3d")
ubm_gmm.plot3D_ellipsoid(num_sigmas=1, ax=ax)

In [ ]:
iv_model = hnp.pdfs.JFATotal(K=num_comp, x_dim=x_dim, y_dim=y_dim)
N=[]
F=[]
for x_i in x:
    N_i, u_x_i = ubm_gmm.accum_suff_stats(x_i)
    N_i, F_i = ubm_gmm.norm_suff_stats(N_i, u_x_i)
    N.append(N_i.reshape(1,-1))
    F.append(F_i.reshape(1,-1))

N = np.concatenate(N, axis=0)
F = np.concatenate(F, axis=0)

elbo, elbo_norm = iv_model.fit(N, F)
fig = plt.figure()
plt.plot(elbo_norm)
plt.xlabel('iters')
plt.ylabel('log(p(x))')
plt.grid(True)

In [ ]:
num_utts=100
w = np.randn(num_utts, 2)
M = ubm_gmm.mu.ravel() + ubm_gmm.cholLambda.ravel() * np.dot(w, iv_model.T)
M = M.reshape(num_utts, num_comp, x_dim)
fig = plt.figure()
ax = fig.add_subplot(projection='3d')
colors = ['b', 'g','r','c','m','y','k','b']
for i in range(num_comp):
    ax.plot_surface(M[:,i,0], M[:,i,1], M[:,i,2], alpha=0.2, color=colors[i])
    ax.scatter(M[:,i,0], M[:,i,1], M[:,i,2], marker='o', color=colors[i])
ax.set_xlabel('x1')
ax.set_ylabel('x2')
ax.set_zlabel('x3')
plt.show()


